In [1]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

import tweepy
import pickle
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import twitter
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tania\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Data from TFM_fake_news github
df = pd.read_csv("Map1.csv",encoding= 'unicode_escape', sep=";")
df.head()

,topic,text,headline,words_h,word_size_h,avg_syllables_word_h,unique_words_h,ttr_h,mltd_h,sents,...,propn_ratio,noun_ratio,adp_ratio,det_ratio,punct_ratio,pron_ratio,verb_ratio,adv_ratio,sym_ratio,label
0,Science,la nasa recupera el contacto con un satélite d...,la nasa recupera el contacto con un satélite d...,16,538,25,875,9375,7168,16,...,48,2443,1775,1566,71,167,835,313,0,1
1,Economy,amlo aceleraría el consumo y el crecimiento ec...,amlo aceleraría el consumo y el crecimiento ec...,11,527,255,8182,9091,3388,5,...,922,2282,1553,165,1117,49,631,97,0,1
2,Sport,al borde de un colapso nervioso quedó el hábil...,compañero de james se calvea y le juega pesa...,12,375,167,1000,1000,0,13,...,734,1658,1522,1332,1304,435,815,353,54,0
3,Politics,"mediante pupitrazo de último minuto anoche, el...",dian gravará este año a los niños que recojan ...,12,458,192,1000,1000,0,8,...,223,223,1933,1524,1004,74,669,335,0,0
4,Politics,muy temprano esta mañana el expresidente y aho...,uribe asegura que insultó 358 guerrilleros,6,617,25,1000,1000,0,9,...,577,1731,846,1077,200,538,1038,50,77,0


In [3]:
df1 = df.iloc[:500]
df1

,topic,text,headline,words_h,word_size_h,avg_syllables_word_h,unique_words_h,ttr_h,mltd_h,sents,...,propn_ratio,noun_ratio,adp_ratio,det_ratio,punct_ratio,pron_ratio,verb_ratio,adv_ratio,sym_ratio,label
0,Science,la nasa recupera el contacto con un satélite d...,la nasa recupera el contacto con un satélite d...,16,538,25,875,9375,7168,16,...,48,2443,1775,1566,71,167,835,313,0,1
1,Economy,amlo aceleraría el consumo y el crecimiento ec...,amlo aceleraría el consumo y el crecimiento ec...,11,527,255,8182,9091,3388,5,...,922,2282,1553,165,1117,49,631,97,0,1
2,Sport,al borde de un colapso nervioso quedó el hábil...,compañero de james se calvea y le juega pesa...,12,375,167,1000,1000,0,13,...,734,1658,1522,1332,1304,435,815,353,54,0
3,Politics,"mediante pupitrazo de último minuto anoche, el...",dian gravará este año a los niños que recojan ...,12,458,192,1000,1000,0,8,...,223,223,1933,1524,1004,74,669,335,0,0
4,Politics,muy temprano esta mañana el expresidente y aho...,uribe asegura que insultó 358 guerrilleros,6,617,25,1000,1000,0,9,...,577,1731,846,1077,200,538,1038,50,77,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Politics,el 57% de catalanes cree que puigdemont ganó e...,el 57% de catalanes cree que puigdemont ganó e...,18,428,167,6667,7778,2268,14,...,807,146,1472,1613,1165,218,743,282,0,1
496,Politics,las fuerzas leales al gobierno de sudán del su...,amnistía acusa a las fuerzas del gobierno surs...,16,469,20,875,9375,7168,28,...,525,2059,1796,1453,106,121,928,242,4,1
497,Politics,la consejera de fomento del gobierno de castil...,"fomento pide ""un cambio de rumbo"" en la políti...",14,393,164,8571,9286,5488,12,...,482,1969,1995,1343,952,196,926,30,0,1
498,Entertainment,andrea legarreta explota al recordar polémica ...,andrea legarreta explota al recordar polémica ...,8,60,262,1000,1000,0,15,...,439,152,1257,936,1257,906,1667,614,0,1


In [4]:
#### Creating a function to convert the text in lowercase, remove the extra space, special chr., ulr and links.


In [5]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [6]:
df["text"] = df["text"].apply(wordopt)

In [7]:
x = df["text"]
y = df["label"]

In [8]:
#### Splitting the dataset into training set and testing set. 

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state= 8)

In [10]:
#### Convert text to vectors

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [13]:
### 1. Logistic Regression

In [14]:
from sklearn.linear_model import LogisticRegression

In [15]:
LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [16]:
pred_lr=LR.predict(xv_test)

In [17]:
LR.score(xv_test, y_test)

0.9195278969957081

In [18]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92       471
           1       0.92      0.92      0.92       461

    accuracy                           0.92       932
   macro avg       0.92      0.92      0.92       932
weighted avg       0.92      0.92      0.92       932



In [19]:
### 2. Decision Tree Classification

In [20]:
from sklearn.tree import DecisionTreeClassifier

In [21]:
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [22]:
pred_dt = DT.predict(xv_test)

In [23]:
DT.score(xv_test, y_test)

0.9002145922746781

In [24]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       0.91      0.89      0.90       471
           1       0.89      0.91      0.90       461

    accuracy                           0.90       932
   macro avg       0.90      0.90      0.90       932
weighted avg       0.90      0.90      0.90       932



In [25]:
### 3. Gradient Boosting Classifier

In [26]:
from sklearn.ensemble import GradientBoostingClassifier

In [27]:
GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [28]:
pred_gbc = GBC.predict(xv_test)

In [29]:
GBC.score(xv_test, y_test)

0.944206008583691

In [30]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       0.97      0.92      0.94       471
           1       0.92      0.97      0.95       461

    accuracy                           0.94       932
   macro avg       0.95      0.94      0.94       932
weighted avg       0.95      0.94      0.94       932



In [33]:
### 4. Random Forest Classifier

In [34]:
from sklearn.ensemble import RandomForestClassifier

In [35]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [36]:
pred_rfc = RFC.predict(xv_test)

In [37]:
RFC.score(xv_test, y_test)

0.9495708154506438

In [38]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.93      0.97      0.95       471
           1       0.97      0.93      0.95       461

    accuracy                           0.95       932
   macro avg       0.95      0.95      0.95       932
weighted avg       0.95      0.95      0.95       932



In [39]:
# Model Testing With Manual Entry

### News

In [40]:
import pyttsx3  
s = pyttsx3.init()
data = "Fake New"
data1 = "Not A Fake New"  
def output_lable(n):
    if n == 0:
        
        s.say(data)  
        s.runAndWait()
        return "Fake News"
    elif n == 1:
             
        s.say(data1)  
        s.runAndWait() 
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]), 
                                                                                                              output_lable(pred_DT[0]), 
                                                                                                              output_lable(pred_GBC[0]), 
                                                                                                              output_lable(pred_RFC[0])))

In [41]:
news = str(input())
manual_testing(news)

Evo morales es un feo


LR Prediction: Fake News 
DT Prediction: Fake News 
GBC Prediction: Fake News 
RFC Prediction: Fake News


In [42]:
### Analysis

In [43]:
dftext=df1.iloc[0:len(df)]['text']
dftext

0      la nasa recupera el contacto con un satélite d...
1      amlo aceleraría el consumo y el crecimiento ec...
2      al borde de un colapso nervioso quedó el hábil...
3      mediante pupitrazo de último minuto anoche  el...
4      muy temprano esta mañana el expresidente y aho...
                             ...                        
495    el   de catalanes cree que puigdemont ganó el ...
496    las fuerzas leales al gobierno de sudán del su...
497    la consejera de fomento del gobierno de castil...
498    andrea legarreta explota al recordar polémica ...
499         el entrenador del manchester city  pep gu...
Name: text, Length: 500, dtype: object

In [44]:
import nltk
from nltk import tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [45]:
#This splits all the sentences up which makes it easier for us to work with.
all_sentences = []

for word in dftext:
    all_sentences.append(word)
all_sentences

lines = list()
for line in all_sentences:    
    words = line.split()
    for w in words: 
       lines.append(w)
print(lines)

['la', 'nasa', 'recupera', 'el', 'contacto', 'con', 'un', 'satélite', 'desaparecido', 'en', 'número', 'gracias', 'a', 'un', 'astrónomo', 'aficionado', 'la', 'nave', 'image', 'fue', 'lanzada', 'el', 'número', 'de', 'marzo', 'del', 'año', 'número', 'y', 'el', 'contacto', 'se', 'había', 'perdido', 'inesperadamente', 'el', 'número', 'de', 'diciembre', 'de', 'número', 'la', 'nasa', 'ha', 'recuperado', 'el', 'contacto', 'perdido', 'hace', 'número', 'años', 'con', 'el', 'satélite', 'image', 'cuya', 'misión', 'era', 'tomar', 'imágenes', 'de', 'la', 'magnetosfera', 'de', 'la', 'tierra', 'gracias', 'a', 'la', 'ayuda', 'de', 'un', 'astrónomo', 'aficionado', 'a', 'mediados', 'de', 'enero', 'este', 'astrónomo', 'cuyo', 'nombre', 'no', 'ha', 'sido', 'revelado', 'informó', 'de', 'que', 'creía', 'haber', 'tomado', 'contacto', 'con', 'este', 'satélite', 'de', 'la', 'agencia', 'espacial', 'y', 'aeronáutica', 'estadounidense', 'image', 'imager', 'for', 'magnetopause', 'to', 'aurora', 'global', 'explorati

In [46]:
#Removing Punctuation
lines = [re.sub(r'[^A-Za-z0-9]+', '', x) for x in lines]
lines
lines2 = []
for word in lines:
    if word != '':
        lines2.append(word)

In [47]:
#This is stemming the words to their root
from nltk.stem.snowball import SnowballStemmer

# The Snowball Stemmer requires that you pass a language parameter
s_stemmer = SnowballStemmer(language='spanish')
stem = []
for word in lines2:
    stem.append(s_stemmer.stem(word))
    
stem

['la',
 'nas',
 'recuper',
 'el',
 'contact',
 'con',
 'un',
 'satlit',
 'desaparec',
 'en',
 'nmer',
 'graci',
 'a',
 'un',
 'astrnom',
 'aficion',
 'la',
 'nav',
 'imag',
 'fue',
 'lanz',
 'el',
 'nmer',
 'de',
 'marz',
 'del',
 'ao',
 'nmer',
 'y',
 'el',
 'contact',
 'se',
 'hab',
 'perd',
 'inesper',
 'el',
 'nmer',
 'de',
 'diciembr',
 'de',
 'nmer',
 'la',
 'nas',
 'ha',
 'recuper',
 'el',
 'contact',
 'perd',
 'hac',
 'nmer',
 'aos',
 'con',
 'el',
 'satlit',
 'imag',
 'cuy',
 'misin',
 'era',
 'tom',
 'imgen',
 'de',
 'la',
 'magnetosfer',
 'de',
 'la',
 'tierr',
 'graci',
 'a',
 'la',
 'ayud',
 'de',
 'un',
 'astrnom',
 'aficion',
 'a',
 'medi',
 'de',
 'ener',
 'este',
 'astrnom',
 'cuy',
 'nombr',
 'no',
 'ha',
 'sid',
 'revel',
 'inform',
 'de',
 'que',
 'cre',
 'hab',
 'tom',
 'contact',
 'con',
 'este',
 'satlit',
 'de',
 'la',
 'agenci',
 'espacial',
 'y',
 'aeronut',
 'estadounidens',
 'imag',
 'imag',
 'for',
 'magnetopaus',
 'to',
 'auror',
 'global',
 'exploration',

In [48]:
# Geeting rid of the stopwords.
stem2 = []

for word in stem:
    if word not in stopwords.words('spanish'):
        stem2.append(word)

stem2

['nas',
 'recuper',
 'contact',
 'satlit',
 'desaparec',
 'nmer',
 'graci',
 'astrnom',
 'aficion',
 'nav',
 'imag',
 'lanz',
 'nmer',
 'marz',
 'ao',
 'nmer',
 'contact',
 'hab',
 'perd',
 'inesper',
 'nmer',
 'diciembr',
 'nmer',
 'nas',
 'recuper',
 'contact',
 'perd',
 'hac',
 'nmer',
 'aos',
 'satlit',
 'imag',
 'cuy',
 'misin',
 'tom',
 'imgen',
 'magnetosfer',
 'tierr',
 'graci',
 'ayud',
 'astrnom',
 'aficion',
 'medi',
 'ener',
 'astrnom',
 'cuy',
 'nombr',
 'sid',
 'revel',
 'inform',
 'cre',
 'hab',
 'tom',
 'contact',
 'satlit',
 'agenci',
 'espacial',
 'aeronut',
 'estadounidens',
 'imag',
 'imag',
 'for',
 'magnetopaus',
 'to',
 'auror',
 'global',
 'exploration',
 'lanz',
 'nmer',
 'marz',
 'nmer',
 'contact',
 'perdi',
 'inesper',
 'nmer',
 'diciembr',
 'nmer',
 'trat',
 'determin',
 'si',
 'seal',
 'proven',
 'imag',
 'goddard',
 'spac',
 'flight',
 'cent',
 'coordin',
 'uso',
 'cinc',
 'anten',
 'separ',
 'par',
 'adquir',
 'seal',
 'radiofrecuent',
 'nav',
 'part',
 

In [98]:
#Counting the left words.
dftext2 = pd.DataFrame(stem2)
dftext2 = dftext2[0].value_counts()
dftext2

par          2310
com          1279
ms           1130
pas           816
tod           801
             ... 
trast           1
cors            1
hemerotec       1
svq             1
jeremy          1
Name: 0, Length: 14327, dtype: int64

In [99]:
#This will give frequencies of our words

from nltk.probability import FreqDist
freqdoctor = FreqDist()

for words in dftext2:
    freqdoctor[words] += 1

freqdoctor

FreqDist({1: 6149, 2: 2106, 3: 1147, 4: 744, 5: 550, 6: 389, 7: 309, 8: 230, 9: 198, 10: 169, ...})

In [ ]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#This is a simple plot that shows the top 20 words being used

dftext2 = dftext2[:20,]
plt.figure(figsize=(10,5))
sns.barplot(dftext2.values, dftext2.index, alpha=0.8)
plt.title('Top Words Overall')
plt.ylabel('Word from Tweet', fontsize=12)
plt.xlabel('Count of Words', fontsize=12)
plt.show()

In [ ]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [ ]:
def show_ents(doc):
    if doc.ents:
        for ent in doc.ents:
            print(ent.text + ' - ' + ent.label_ + ' - ' + str(spacy.explain(ent.label_)))

In [ ]:
str1 = " " 
stem2 = str1.join(lines2)
stem2 = nlp(stem2)

label = [(X.text, X.label_) for X in stem2.ents]

df6 = pd.DataFrame(label, columns = ['Word','Entity'])
df7 = df6.where(df6['Entity'] == 'ORG')
df7 = df7['Word'].value_counts()

In [ ]:
df1 = df7[:20,]
plt.figure(figsize=(10,5))
sns.barplot(df1.values, df1.index, alpha=0.8)
plt.title('Top Organizations Mentioned')
plt.ylabel('Word from Tweet', fontsize=12)
plt.xlabel('Count of Words', fontsize=12)
plt.show()

In [ ]:
str1 = " " 
stem2 = str1.join(lines2)

stem2 = nlp(stem2)

label = [(X.text, X.label_) for X in stem2.ents]

df10 = pd.DataFrame(label, columns = ['Word','Entity'])

df10 = df10.where(df10['Entity'] == 'PERSON')

df11 = df10['Word'].value_counts()

In [ ]:
df1 = df11[:20,]
plt.figure(figsize=(10,5))
sns.barplot(df1.values, df1.index, alpha=0.8)
plt.title('Top People Mentioned')
plt.ylabel('Word from Tweet', fontsize=12)
plt.xlabel('Count of Words', fontsize=12)
plt.show()